In [36]:
import re
import torch
import json
import requests
import ebooklib
from ebooklib import epub
import urllib

from bs4 import BeautifulSoup
from internetarchive import search_items, get_item, Search

from seqeval.metrics import accuracy_score, f1_score, classification_report, precision_score, recall_score
from seqeval.scheme import BILOU
from tokenizers import decoders, Encoding
from transformers import BertForTokenClassification, BertTokenizer, BertTokenizerFast, BatchEncoding

from ner_pipeline.alignment import align_tokens_and_annotations_bilou
from ner_pipeline.scrape_for_training import do_search, prepare_data
# from ner_pipeline.prepare_data import prepare_data
from ner_pipeline.labelset import LabelSet

In [37]:
model = BertForTokenClassification.from_pretrained("bert_ner_il_od-with-gpu-100.model")

In [3]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [4]:
label_list = ['O', 'B-CLEntity', 'I-CLEntity', 'L-CLEntity', 'U-CLEntity']

In [5]:
# Do search
il_od: str = "iliad OR odyssey AND mediatype:texts"  # 771,646 with full_text_search, 6240 without
search_res: Search = do_search(keyword_string=il_od)

Search string: iliad OR odyssey AND mediatype:texts
Results: 538884


In [6]:
def get_epub_url(book_url: str) -> str:
    """Scrape the epub file url from the web page 
    directed by the input ``book_url``.
    """
    book_web_page = requests.get(book_url)
    book_web_page_soup = BeautifulSoup(book_web_page.text, 'html.parser')
    epub_url = ""
    for link in book_web_page_soup.find_all('a', 'stealth'):
        epub_url_candidate = link.get('href')
        if epub_url_candidate.endswith('.epub'):
            epub_url = epub_url_candidate   
    return epub_url

In [10]:
def read_epub_from_url(epub_url):
    with urllib.request.urlopen(epub_url) as url:
        s = url.read()
    with open(r"c:\tmp\test.epub", "wb") as f:
        f.write(s)
    epub_book = epub.read_epub(r"c:\tmp\test.epub")
    return epub_book

In [15]:
# index = 0
# for item in search_res:
#     if index == 0:
#         book_url = "https://archive.org/details/" + item["fields"]["identifier"][0]
#         epub_url = get_epub_url(book_url)
#         if epub_url:
#             epub_url = 'https://archive.org' + epub_url
#             # Access to the epub file
#             epub_book = read_epub_from_url(epub_url)
#             print("type: ", type(epub_book))
#             chapters = []
#             for item in book.get_items():
#                 if item.get_type() == ebooklib.ITEM_DOCUMENT:
#                     chapters.append(item.get_content())
#             print("chapters: ", chapters)
#             soup1 = BeautifulSoup(chapters[1], 'html.parser')
#             find = [para for para in soup1.find_all(['p','span'])]
            
#             print(find)
#             print(find[0].get_text())
#             print(find[20].attrs['id'])
#             print(find[2000])
#             print(find[20])
            
#             index += 1
#     else:
#         break

In [20]:
book = epub.read_epub("epub_downloads/introductionsto00cole_introductionsto00cole.epub")
type(book)

ebooklib.epub.EpubBook

In [35]:
# Get ITEM_DOCUMENT
chapters = []
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        chapters.append(item.get_content())

soup1 = BeautifulSoup(chapters[2], 'html.parser')
find = [para for para in soup1.find_all(['p','span'])]
# print(find)
# print(find[0].get_text())
# print(find[23].attrs['id'])
# print(find[2000])
# print(find[20])

15


In [9]:
pattern = r'Iliad\s\d{1,2}\.\d{1,4}|Il\.*\s\d{1,2}\.\d{1,4}|Iliad\s.[ivxlcdm]*\.\s*\d{1,4}| \
            Il\.*\s.[ivxlcdm]*\.\s*\d{1,4}|book\s*.[ivxlcdm]\.\sline\s*\d{1,4}| \
            Odyssey\s\d{1,2}\.\d{1,4}|Od\.*\s\d{1,2}\.\d{1,4}|Odyssey\s.[ivxlcdm]*\.\s*\d{1,4}| \
            Od\.*\s.[ivxlcdm]*\.\s*\d{1,4}'